In [1]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rand

In [19]:
def check_converge(z_0, treshold=2, max_iterations=25):
    """
    Checks if the complex number z_0 converges within the max_iterations.
    """

    z = z_0
    counter = 0
    while True:
        z = z ** 2 + z_0
        counter += 1

        if abs(z) > 2:
            return False
        if counter > max_iterations:
            return True

def determine_area(i = 25, s=10000, sample_method="pure"):
    """
    Determine the area of the mandelbrot set.

    Parameters:
    i: int, Max number of iterations
    s: int, The number of sample points.
    sample_method: string, options are:
        'pure': pure random sampling,
        'hypercube': hypercube sampling,
        'orthogonal': orthogonal sampling
    """

    xmin = -2
    xmax = 1
    ymin = -2
    ymax = 2
    bounding_box_area = (xmax - xmin) * (ymax - ymin)

    if sample_method == "pure":
        Z = generate_sample_points_pure_random(s, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
    elif sample_method == "hypercube":
        Z = generate_sample_points_hypercube(s, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
    elif sample_method == "orthogonal":
        Z = generate_sample_points_orthogonal(s, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

    Z0 = Z

    N = np.zeros(s)

    for _ in range(i):
        Z = np.where(abs(Z) < 2, Z ** 2 + Z0, 100)

    N[Z != 100] = 1

    n_good_points = sum(N)
    
    return bounding_box_area * (n_good_points / s)

def plot_i_s_contour(i_list, s_list, log=False):
    """
    Plots the difference between the expected and calculated area for different values for i and s. 
    i is the maximum number of iterations and s is the number of samples. 
    """

    total_evaluations = len(i_list) * len(s_list)
    n_evaluations = 0

    expected_area = 1.506484

    data = []
    for i in i_list:

        results = []
        for s in s_list:
            results.append(determine_area(i=i, s=s) - expected_area)
            n_evaluations += 1

            if log and n_evaluations % 10 == 0:
                print(f"Progress: {n_evaluations/total_evaluations * 100:0f}%", end="\r")
        
        data.append(results)

    contour = plt.contourf(s_list, i_list, data, 10, cmap="PuBuGn_r")
    
    colorbar = plt.colorbar(contour)
    colorbar.set_label("Difference between expected area")

    plt.xlabel("Samples")
    plt.ylabel("Max iterations")

    plt.xscale("log")
    plt.yscale("log")
    plt.show()

def plot_i_s_variance(i_list, s_list, nruns=10, log=False):
    """
    Plots the variance of the difference between the expected and calculated area for different values for i and s. 
    i is the maximum number of iterations and s is the number of samples. 
    """
    
    total_evaluations = len(i_list) * len(s_list) * nruns
    n_evaluations = 0

    expected_area = 1.506484

    data = []
    for i in i_list:

        results = []
        for s in s_list:

            run_results = []
            for run in range(nruns):
                run_results.append(determine_area(i=i, s=s))
                n_evaluations += 1

                if log and n_evaluations % 10 == 0:
                    print(f"Progress: {n_evaluations/total_evaluations * 100:1f}%", end="\r")

            results.append(np.var(run_results))
        
        data.append(results)

    contour = plt.contourf(s_list, i_list, data, 10, cmap="PuBuGn_r")
    
    colorbar = plt.colorbar(contour)
    colorbar.set_label("Variance in area")

    plt.xlabel("Samples")
    plt.ylabel("Max iterations")

    plt.xscale("log")
    plt.yscale("log")
    plt.show()

def generate_sample_points_pure_random(s=10000, xmin=-2, xmax=1, ymin=-2, ymax=2):
    
    X = rand.rand(s) * (xmax - xmin) + xmin
    Y = rand.rand(s) * (ymax - ymin) + ymin

    return X + Y * 1j

def generate_sample_points_hypercube(s=10000, xmin=-2, xmax=1, ymin=-2, ymax=2):
    sampler = stats.qmc.LatinHypercube(2)
    points = sampler.random(s)

    l_bounds = [xmin, ymin]
    u_bounds = [xmax, ymax]

    points = stats.qmc.scale(points, l_bounds, u_bounds)

    return points[:,0] + points[:,1] * 1j

def generate_sample_points_orthogonal(s=10000, xmin=-2, xmax=1, ymin=-2, ymax=2):
    major = 9999
    samples = major ** 2

    xlist = np.zeros((major, major))
    ylist = np.zeros((major, major))

    m = 0
    runs = 10
    scale = 4 / samples

    x_list = []
    y_list = []

    for i in range(major):
        for j in range(major):
            m += 1
            xlist[i][j] = m 
            ylist[i][j] = m

    for k in range(runs):
        print(k)
        for i in range(major):
            permute(xlist[i], major)
            permute(ylist[i], major)

        for i in range(major):
            for j in range(major):
                x = -2 + scale * xlist[i][j] + np.random.rand()
                y = -2 + scale * ylist[i][j] + np.random.rand() 

                x_list.append(x)
                y_list.append(y)

    return x_list, y_list




def permute(list, N):
    for i in reversed(range(1, N)):
        h = list[i]
        r = np.random.randint(0, high=i)
        list[i] = list[r]
        list[r] = h



    


In [3]:
#pure = determine_area(i = 2500, s=10000000, sample_method="pure")


In [4]:
#hyper = determine_area(i = 2500, s=10000000, sample_method="hypercube")

In [5]:
#print(pure, hyper)

In [6]:
#borders = -0.743643135 + 0.121825963j, 0.000014628
